# Self-Drive Cars' Sentiment Analysis

- Labelling data



# 1)-Importing key modules

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import datetime as dt
import numpy as np
import requests
import pickle

In [2]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem.porter import PorterStemmer

from nltk.corpus import stopwords
nltk.download('stopwords')
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hassan.sherwani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import spacy 

nlp = spacy.load("en_core_web_sm") # preload model

In [4]:
# Visualization 
import matplotlib.pyplot as plt
import seaborn as sns
from pprint import pprint
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 2)-Loading data

In [5]:
filename='train.pkl'

In [6]:
train=pd.read_pickle(filename)
train.shape

(15000, 1)

In [7]:
train.head(2)

,TEXT
143880,"""Over the years, the MLK estate has also licen..."
87455,Porsche celebrates its racing history with the...


# 4)- Clean data

In [11]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hassan.sherwani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [12]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;#]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z +_]')
STOPWORDS = stopwords.words('english')
STOPWORDS.extend(['rt', 'http']) # extend stopwords; rt means re-tweet
STOPWORDS = set(STOPWORDS)

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join([word for word in text.split() if word not in STOPWORDS]) # delete stopwords from text
    text = text.strip()
    return text

In [13]:
train['cleaned']=[text_prepare(x) for x in train['TEXT']]

# 5)-Sentiments extraction

In [16]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\hassan.sherwani\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [17]:
plt.rcParams['figure.figsize'] = (6.0, 6.0)
sid= SentimentIntensityAnalyzer()

In [18]:
train['Sent_compound_Polarity'] = train.TEXT.apply(lambda x:sid.polarity_scores(x)['compound'])
train['neutral']= train.TEXT.apply(lambda x:sid.polarity_scores(x)['neu'])
train['risk']= train.TEXT.apply (lambda x:sid.polarity_scores(x)['neg'])
train['benefit']= train.TEXT.apply(lambda x:sid.polarity_scores(x)['pos'])
train['Sent_type']=''

In [19]:
train.head()

,TEXT,cleaned,Sent_compound_Polarity,neutral,risk,benefit,Sent_type
143880,"""Over the years, the MLK estate has also licen...",years mlk estate also licensed king speeches u...,0.6166,0.910,0.000,0.090,
87455,Porsche celebrates its racing history with the...,porsche celebrates racing history 911 carrera ...,0.5719,0.778,0.000,0.222,
103986,"Hmm, could that be the Mercedes? It made a ver...",hmm could mercedes made strange noise hamilton...,-0.2716,0.884,0.116,0.000,
261437,The Full Carbon Audi RS6 is simply Amazing pi...,full carbon audi rs6 simply amazing pictwitter...,0.5859,0.678,0.000,0.322,
205444,Vijay Mayadas explores #blockchain potential i...,vijay mayadas explores blockchain potential ch...,0.3612,0.884,0.000,0.116,


In [20]:
train.loc[train.Sent_compound_Polarity > 0, 'Sent_type']= 'Benefit'

train.loc[train.Sent_compound_Polarity == 0, 'Sent_type']= 'Neutral'

train.loc[train.Sent_compound_Polarity < 0, 'Sent_type']= 'Risky'

In [21]:
train.head()

,TEXT,cleaned,Sent_compound_Polarity,neutral,risk,benefit,Sent_type
143880,"""Over the years, the MLK estate has also licen...",years mlk estate also licensed king speeches u...,0.6166,0.910,0.000,0.090,Benefit
87455,Porsche celebrates its racing history with the...,porsche celebrates racing history 911 carrera ...,0.5719,0.778,0.000,0.222,Benefit
103986,"Hmm, could that be the Mercedes? It made a ver...",hmm could mercedes made strange noise hamilton...,-0.2716,0.884,0.116,0.000,Risky
261437,The Full Carbon Audi RS6 is simply Amazing pi...,full carbon audi rs6 simply amazing pictwitter...,0.5859,0.678,0.000,0.322,Benefit
205444,Vijay Mayadas explores #blockchain potential i...,vijay mayadas explores blockchain potential ch...,0.3612,0.884,0.000,0.116,Benefit


In [22]:
train.Sent_type.value_counts()

Neutral    6513
Benefit    5912
Risky      2575
Name: Sent_type, dtype: int64

In [23]:
train_df=train.drop(['cleaned','Sent_compound_Polarity','neutral','risk','benefit'], axis=1)
train_df.head(2)

,TEXT,Sent_type
143880,"""Over the years, the MLK estate has also licen...",Benefit
87455,Porsche celebrates its racing history with the...,Benefit


In [24]:
#train_df.to_pickle('model.pkl')
#train_df.to_csv('model.csv')

# For Full data 

In [25]:
filename='tweets.csv'

In [26]:
df = pd.read_csv(filename, delimiter='\t', header=None,names=['TWEET ID','TIMESTAMP','UTC DATE','USER ID','SCREEN NAME','NAME','Q','TEXT'])
df.shape

(455728, 8)

In [27]:
df_label=df[['TEXT','UTC DATE']]
df_label=df_label.drop(df_label.index[0])

In [28]:
df_label.shape

(455727, 2)

In [29]:
df_label.head()

,TEXT,UTC DATE
1,"Another reason I failed in love is I'm poor,I ...",2018-07-31 23:13:47
2,2019 Mercedes-Benz C 300 and AMG C 43 First Dr...,2018-07-31 23:30:00
3,"Happy birthday to the Audi chasing maniac, hop...",2018-07-31 22:52:45
4,Come see Mercedes Benz stadium in Atlanta...,2018-07-31 23:02:25
5,Italy took Cristiano and ever since then they’...,2018-07-31 22:53:15


### for descriptive analysis

In [30]:
df_label['Sent_compound_Polarity'] = df_label.TEXT.apply(lambda x:sid.polarity_scores(x)['compound'])
df_label['neutral']= df_label.TEXT.apply(lambda x:sid.polarity_scores(x)['neu'])
df_label['risk']= df_label.TEXT.apply (lambda x:sid.polarity_scores(x)['neg'])
df_label['benefit']= df_label.TEXT.apply(lambda x:sid.polarity_scores(x)['pos'])
df_label['Sent_type']=''

In [31]:
df_label.head()

,TEXT,UTC DATE,Sent_compound_Polarity,neutral,risk,benefit,Sent_type
1,"Another reason I failed in love is I'm poor,I ...",2018-07-31 23:13:47,0.9246,0.638,0.057,0.305,
2,2019 Mercedes-Benz C 300 and AMG C 43 First Dr...,2018-07-31 23:30:00,0.0000,1.000,0.000,0.000,
3,"Happy birthday to the Audi chasing maniac, hop...",2018-07-31 22:52:45,0.9100,0.448,0.087,0.465,
4,Come see Mercedes Benz stadium in Atlanta...,2018-07-31 23:02:25,0.0000,1.000,0.000,0.000,
5,Italy took Cristiano and ever since then they’...,2018-07-31 22:53:15,0.0000,1.000,0.000,0.000,


In [32]:
df_label.loc[df_label.Sent_compound_Polarity > 0, 'Sent_type']= 'Benefit'

df_label.loc[df_label.Sent_compound_Polarity == 0, 'Sent_type']= 'Neutral'

df_label.loc[df_label.Sent_compound_Polarity < 0, 'Sent_type']= 'Risky'

In [33]:
df_label.head()

,TEXT,UTC DATE,Sent_compound_Polarity,neutral,risk,benefit,Sent_type
1,"Another reason I failed in love is I'm poor,I ...",2018-07-31 23:13:47,0.9246,0.638,0.057,0.305,Benefit
2,2019 Mercedes-Benz C 300 and AMG C 43 First Dr...,2018-07-31 23:30:00,0.0000,1.000,0.000,0.000,Neutral
3,"Happy birthday to the Audi chasing maniac, hop...",2018-07-31 22:52:45,0.9100,0.448,0.087,0.465,Benefit
4,Come see Mercedes Benz stadium in Atlanta...,2018-07-31 23:02:25,0.0000,1.000,0.000,0.000,Neutral
5,Italy took Cristiano and ever since then they’...,2018-07-31 22:53:15,0.0000,1.000,0.000,0.000,Neutral


In [34]:
df_label.shape

(455727, 7)

In [35]:
df_label=df_label.drop(['Sent_compound_Polarity','neutral','risk','benefit'], axis=1)
df_label.head(2)

,TEXT,UTC DATE,Sent_type
1,"Another reason I failed in love is I'm poor,I ...",2018-07-31 23:13:47,Benefit
2,2019 Mercedes-Benz C 300 and AMG C 43 First Dr...,2018-07-31 23:30:00,Neutral


In [36]:
df_label.Sent_type.value_counts()

Neutral    195719
Benefit    182533
Risky       77475
Name: Sent_type, dtype: int64

**save as descriptive**

In [37]:
df_label.to_pickle('descript.pkl')

In [39]:
df_label.to_excel('descript.xlsx')

# END OF NOTEBOOK